<a href="https://colab.research.google.com/github/jimin0/Graduation_Project/blob/main/ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 176), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.68 MiB | 6.82 MiB/s, done.
Resolving deltas: 100% (882/882), done.
/content/SEAL-Python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 6.2 MB/s eta 0:00:00
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/SEAL-Python/SEAL'...
Cloning into '/content/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- T

In [ ]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

# 다항식 모듈러스의 차수를 설정합니다.
# 이는 암호화된 다항식의 최대 차수를 정의하고,
# 암호화의 보안 수준과 성능에 영향을 미칩니다.
"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))


# 설정된 암호화 매개변수를 기반으로 SEAL 컨텍스트를 생성합니다.
# 이 컨텍스트는 암호화, 복호화, 키 생성 등에 사용됩니다.
context = SEALContext(parms)


# 계수 모듈러스를 설정합니다. 이는 암호화 시 사용되는 모듈러스 값으로,
# BFVDefault는 보안 수준에 맞는 기본 모듈러스 값을 선택합니다.
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# 평문 모듈러스를 설정합니다.
# 이 모듈러스는 암호화되지 않은 데이터가 취할 수 있는 값의 범위를 정의합니다.
# 이 경우, 값의 범위는 0에서 255입니다.
parms.set_plain_modulus(256)



# """
# plaintext modulus는 양의 정수여야 합니다.
# plaintext modulu는 plaintext 데이터 타입의 크기를 결정하며, 곱셈에서 노이즈 예산의 소비를 결정합.
# 따라서, 최상의 성능을 위해 가능한 한 plaintext 데이터 타입을 작게 유지하는 것이 중요함.

# (!) 각각의 암호화된 값은  noise budget 을 가지고 있음. 암호문에 대한 연산을 할 때마다  noise budget이 소모됨.
# 한 암호문의 노이즈 예산이 0에 도달하면 그 암호문은 너무 손상되어 복호화할 수 없게 된다함.

# """


In [ ]:
# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)


batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2
print(f'Plaintext matrix row size: {row_size}')

Plaintext matrix row size: 4096.0



TODO
*   곱셈 결과 범위가 256이하인 연산 테스트해보기
*   연속 계산할 경우 correctness 확인
*   2*3*4*2*1*... 이렇게 곱셈을 연달아서 여러번 하는 경우 --> bootstrapping 필요할 수 있음





In [ ]:
# x = 2 암호화

x="2"
x_plain = Plaintext(x)

# plain text로 변경
print("intput : x")
print(f'x_plain: {x_plain}')
print("Express x = " + x + " as a plaintext polynomial " +
      x_plain.to_string() + ".")
print(f'x_plain type: {type(x_plain)}')

print()

# 암호화
x_encrypted1 = Ciphertext()
print("암호화")
x_encrypted1 = encryptor.encrypt(x_plain)
print(f'x_encrypted: {x_encrypted1}')
print(f'x_encrypted to string: {x_encrypted.to_string()}')


# 암호화 된거 정보 출력
print("    + type of encrypted1: ", type(x_encrypted1))
print("    + size of freshly encrypted1 x: " + str(x_encrypted1.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted1.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted1.poly_modulus_degree()))
# 노이즈 예산 출력
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted1)) + " bits")

print()

# 복호화
print("복호화")
x_decrypted = Plaintext()
decryptor.decrypt(x_encrypted1,x_decrypted)
print(f'x_decrypted: {x_decrypted}')
print(f'x_decrypted to string: {x_decrypted.to_string()}')
print()



# 원복 비교
print("비교")
if x == x_decrypted.to_string():
    print(f'Correct!:  [x:{x}] == [x_degrypted: {x_decrypted.to_string()}]')
else:
    print(f'Incorrect!:  [x:{x}] != [x_degrypted: {x_decrypted.to_string()}]')

# --------------------------------------------------------------
print("-"*50)
"""
2 x 3

암호문 * 암호문을 해보겠다.

"""
expected_result = str(2*3)

print("intput : 3")
print("식 : 2 x 3")

print("\n암호화")
x_encrypted2 = encryptor.encrypt(Plaintext("3"))
print(f'x_encrypted: {x_encrypted2}')
#print(f'x_encrypted to string: {x_encrypted.to_string()}')

# 암호화 된거 정보 출력
print("    + size of freshly encrypted x: " + str(x_encrypted2.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted2.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted2.poly_modulus_degree()))
# 노이즈 예산 출력
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted2)) + " bits")
print()

# 연산
print("연산")

# 먼저 결과를 저장할 Ciphertext 객체를 생성
twoPthree = Ciphertext()

# 암호문끼리 곱셈을 수행하고, 결과를 'destination'에 저장
twoPthree = evaluator.multiply(x_encrypted1, x_encrypted2)
print("2 x 3" + " as a ciphertext " + str(twoPthree) + ".") #destination
print("    + After addition, size of twoMthree: " + str(twoPthree.size()))
print("    + noise budget after addition: " + str(decryptor.invariant_noise_budget(twoPthree)) + " bits")
print("    + type of twoMthree: ", type(twoPthree))

# 복호화
print("\n복호화")
x_decrypted = Plaintext()
decryptor.decrypt(twoPthree,x_decrypted)
print(f'x_decrypted: {x_decrypted}')
print(f'x_decrypted to string: {x_decrypted.to_string()}')
print()

# 원복 비교
print("비교")
if expected_result == x_decrypted.to_string():
    print(f'Correct!:  [x:{expected_result}] == [x_degrypted: {x_decrypted.to_string()}]')
else:
    print(f'Incorrect!:  [x:{expected_result}] != [x_degrypted: {x_decrypted.to_string()}]')

# --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4

암호문 * plaintext해볼 예정 -> 실패
"""
expected_result = str(2*3*4)

# print("intput : 4 -> 실패 다음 블록 보세요")
# print("식 : 2 x 3 x 4")

# x_palin = Plaintext("4")


# # 연산
# print("\n연산")
# SixPfour = Ciphertext()

# # 암호문과 plaintext 연산, 결과는 암호문
# SixPfour = evaluator.multiply_plain(twoPthree,x_palin)
# print("2 x 3 x 4" + " as a ciphertext " + str(SixPfour) + ".") #destination
# print("    + After addition, size of SixPfour: " + str(SixPfour.size()))
# print("    + noise budget after addition: " + str(decryptor.invariant_noise_budget(SixPfour)) + " bits")
# print("    + type of SixPfour: ", type(SixPfour))

# """

# 암호문 * plaintext 간의 연산은 노이즈가 별로 안 줄어드는 거 같음.

# """

# # 복호화
# print("\n복호화")
# x_decrypted = Plaintext()
# decryptor.decrypt(SixPfour,x_decrypted)
# print(f'x_decrypted: {x_decrypted}')
# print(f'x_decrypted to string: {x_decrypted.to_string()}')
# print()

# # 원복 비교
# print("비교")
# if expected_result == x_decrypted.to_string():
#     print(f'Correct!:  [x:{expected_result}] == [x_degrypted: {x_decrypted.to_string()}]')
# else:
#     print(f'Incorrect!:  [x:{expected_result}] != [x_degrypted: {x_decrypted.to_string()}]')


# --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4

암호문 * 암호문

batch encoder 사용
"""

print("intput : 4")
print("식 : 2 x 3 x 4")


# BatchEncoder 생성
batch_encoder = BatchEncoder(context)
single_value_array = np.array([4])

plain_four = Plaintext()
plain_four = batch_encoder.encode(single_value_array)

print("\n암호화")
x_encrypted3 = Ciphertext()
x_encrypted3 = encryptor.encrypt(plain_four)

print(f'x_encrypted3: {x_encrypted3}')
#print(f'x_encrypted to string: {x_encrypted.to_string()}')

# 암호화 된거 정보 출력
print("    + size of freshly x_encrypted3 x: " + str(x_encrypted3.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted3.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted3.poly_modulus_degree()))
# 노이즈 예산 출력
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted3)) + " bits")
print()

# 연산
print("연산")

# 먼저 결과를 저장할 Ciphertext 객체를 생성
SixPfour = Ciphertext()

# 암호문끼리 곱셈을 수행하고, 결과를 'destination'에 저장
SixPfour = evaluator.multiply(twoPthree, x_encrypted3)
print("2 x 3" + " as a ciphertext " + str(SixPfour) + ".") #destination
print("    + After addition, size of SixPfour: " + str(SixPfour.size()))
print("    + noise budget after addition: " + str(decryptor.invariant_noise_budget(SixPfour)) + " bits")
print("    + type of SixPfour: ", type(SixPfour))

# 복호화
print("\n복호화")
x_decrypted = Plaintext()
decryptor.decrypt(SixPfour,x_decrypted)
print(f'x_decrypted: {x_decrypted}')
print(f'x_decrypted to string: {x_decrypted.to_string()}')


# 복호화된 결과를 다시 정수 배열로 변환
decoded_values = batch_encoder.decode(x_decrypted)
print("Decoded values:", decoded_values)


# 결과 확인 (첫 번째 원소)
decrypted_integer_result = decoded_values[0]
print(f'Decoded first value: {decrypted_integer_result}')

# 원본 결과 비교
expected_result = 2 * 3 * 4  # 24
print("\n비교")
if expected_result == decrypted_integer_result:
    print(f'Correct!:  [Expected: {expected_result}] == [Decrypted: {decrypted_integer_result}]')
else:
    print(f'Incorrect!:  [Expected: {expected_result}] != [Decrypted: {decrypted_integer_result}]')

# --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4 x 5

암호문 * 암호문

batch encoder 사용
"""



intput : x
x_plain: <seal.Plaintext object at 0x7d7bfd8c94b0>
Express x = 2 as a plaintext polynomial 2.
x_plain type: <class 'seal.Plaintext'>

암호화
x_encrypted: <seal.Ciphertext object at 0x7d7bfd8c8a30>
x_encrypted to string: b'^\xa1\x10\x04\x01\x02\x00\x00K\x99\x06\x00\x00\x00\x00\x00(\xb5/\xfd\xa0a\x00\x08\x00,\x14\r^\xfd_\x9ch)\x10\xa8\x1d\xe6D\xae\xfe\x83I\x1f\xb9\xca\xa1\x18S\xc7\xe0\xee\x9b\xefV\xa9\xf5.\x97\xa5\xaf\x0e\x93\x93,o\x84\xe3J\x9a\x94RJ)S\xabh\x99h\x8eh\xbf\xae\x19\xa8{t\xe2\xdd\t\x8f\x95\x15\xfe\xb4\xce\x8e\x1b8\xdc\x9e}\xc3\xb9\xa5\x8f\xcb\xbf\x98ds\x9e_\xff\xc6\xe1i\xb1!\xf6+\x1f9\xa3\xa0l}\x8b\xb8\xa9B\x84\xbfO\x0f\xc0\xf1\xe8 \xf7\xac\x0cP}o\xc0>\r;\x04\xbd\xe3\xf8\xa9;\xa7\x00\xa5H\x7f"\xa3\xa4^\x8c\x7f\xc9t\xc2\x81w\xe9S\xc0\x11w\xdc\xc7WKP\x80C\x87\xf5l\x0c\r\\F\xd2>e\x93&\xe7(\xeaWM\x95\xa2\x8be\xcf**\xc5\xe7\x05\xe3S\xcb\xb3?\x1f\xdaO\t\xbcm\x99\\\xbc\xfa\xe6\xa2\xb3\n\x88o\xd7\xd4$\x8ey>\xeb\xa6l\x0e\x8f\x9f\x8a\x18\x00\xc0\x07\x9ag\x9b\x18\x18\r\x15\xe9]

## Plaintext 인코딩 실험 테스트

In [ ]:
from seal import *
# Plaintext 객체 생성
plain_four_str = Plaintext("4")
print("Plaintext content:", plain_four_str.to_string())

#BatchEncoder 생성
batch_encoder = BatchEncoder(context)

# 인코딩할 데이터
values_to_encode = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 예시 데이터
plain = Plaintext()
plain_test = batch_encoder.encode(values_to_encode)

# 암호화
encrypted = Ciphertext()
encrypted_test = encryptor.encrypt(plain_test)

# 복호화 및 디코딩
decrypted = Plaintext()
decryptor.decrypt(encrypted_test, decrypted)
decoded_values = []
decoded_values = batch_encoder.decode(decrypted)

# 결과 출력
print("Original values:", values_to_encode)
print("Decoded values:", decoded_values)



# 생성된 Plaintext 객체의 내용 출력 : 문자열 4
print("Plaintext content using '4':", plain_four_str, type(plain_four_str))

# BatchEncoder를 사용한 결과 출력 : int 4
print("BatchEncoder encoded content:", values_to_encode[3], type(decoded_values[3]))


# 다항식 내부 출력
print("Plaintext content using '4':", plain_four_str.to_string())
print("BatchEncoder encoded content:", plain_test.to_string())

Plaintext content: 4
Original values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Decoded values: [1 2 3 ... 0 0 0]
Plaintext content using '4': <seal.Plaintext object at 0x7d7bfd7597b0> <class 'seal.Plaintext'>
BatchEncoder encoded content: 4 <class 'numpy.int64'>
Plaintext content using '4': 4
BatchEncoder encoded content: 3824x^8191 + 44A76x^8190 + A0017x^8189 + 8DE98x^8188 + 6CA9Ex^8187 + 2F23x^8186 + 3BD84x^8185 + CDC00x^8184 + EECF8x^8183 + 97DFCx^8182 + 72535x^8181 + CA076x^8180 + 730CAx^8179 + E609Dx^8178 + B0B6x^8177 + 51E5x^8176 + D413Bx^8175 + 2A5BEx^8174 + 1BBAx^8173 + 45CABx^8172 + 8A7Ex^8171 + 323CDx^8170 + AE820x^8169 + ED277x^8168 + 703C1x^8167 + 38A18x^8166 + 1960Bx^8165 + F07BAx^8164 + 8F12Bx^8163 + 2C1FBx^8162 + 9EB6Ex^8161 + 3ACBDx^8160 + C32CBx^8159 + 99E66x^8158 + 51178x^8157 + F050Bx^8156 + C8C03x^8155 + 9BD5Cx^8154 + B6FBAx^8153 + C922x^8152 + 806DFx^8151 + AD78Dx^8150 + 8D241x^8149 + 4A2C8x^8148 + 255F8x^8147 + 25B3Fx^8146 + 1A7FCx^8145 + D9E63x^8144 + E4C8Ax^8143 + D3154x

In [ ]:
def print_vector(vector):
    print('[ ', end='')
    for i in range(0, 8):
        print(vector[i], end=', ')
    print('... ]')


인수로 안하고 리스트에 저장해서 암호화해보기

In [ ]:
# 길이가 slot_count 길이인 리스트 생성하고 0으로 초기화
pod_matrix = [0] * slot_count

# 값 할당
pod_matrix[0] = 2
pod_matrix[1] = 3
pod_matrix[2] = 4
pod_matrix[3] = 2
pod_matrix[4] = 1
pod_matrix[5] = 3
pod_matrix[6] = 1.5
pod_matrix[7] = 3



print(f'pod matrix: {pod_matrix}')
print(f'len pod matrix : {len(pod_matrix)}')  #8192
print()

#plain text로 인코딩
x_plain = batch_encoder.encode(pod_matrix)

# x_plain의 내용을 문자열로 변환하여 출력
print(f'x_plain: {x_plain}')
print(f'x_plain to string: {x_plain.to_string()}')
print()

#암호화
x_encrypted = encryptor.encrypt(x_plain)
print(f'x_encryted : {x_encrypted}')
print("Encoded encrypt text:", x_encrypted.to_string())
print(f'noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)}') # 146
print('-'*50)



# for i in range(1, len(encrypted_elements)):
#   temp_result = Ciphertext()
#   temp_result = evaluator.multiply(result_multi, encrypted_elements[i])
#   evaluator.relinearize_inplace(temp_result, relin_keys)
#   result_multi = temp_result

# # Decrypt and decode the result
# decrypted_result = decryptor.decrypt(result_multi)
# decoded_result = batch_encoder.decode(decrypted_result)

# # Print the final result
# print(f'Decoded result: {decoded_result}')

# x_multiply = evaluator.square(x_encrypted)

# print(f'size of x_squared: {x_squared.size()}')

# print()
# evaluator.relinearize_inplace(x_squared, relin_keys)
# print(f'size of x_squared (after relinearization): {x_squared.size()}')
# print(f'noise budget in x_squared: {decryptor.invariant_noise_budget(x_squared)} bits')
# decrypted_result = decryptor.decrypt(x_squared)
# pod_result = batch_encoder.decode(decrypted_result)
# print_vector(pod_result)
# print('-'*50)

pod matrix: [2, 3, 4, 2, 1, 3, 1.5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# x_encrypted의 제곱과 유사하지만, 서로 다른 두 개의 객체를 곱함
x_multiplied = evaluator.multiply(x_encrypted, )

# 재선형화 수행
evaluator.relinearize_inplace(x_multiplied, relin_keys)

# 결과 크기 및 잡음 예산 출력
print(f'size of x_multiplied: {x_multiplied.size()}')
print(f'noise budget in x_multiplied: {decryptor.invariant_noise_budget(x_multiplied)} bits')

# 복호화 및 결과 출력
decrypted_result_multiplied = decryptor.decrypt(x_multiplied)
pod_result_multiplied = batch_encoder.decode(decrypted_result_multiplied)
print_vector(pod_result_multiplied)


size of x_multiplied: 2
noise budget in x_multiplied: 110 bits
[ 4, 9, 16, 4, 1, 0, 0, 0, ... ]


In [ ]:
encrypted_values = [encryptor.encrypt(batch_encoder.encode([2])),
                    encryptor.encrypt(batch_encoder.encode([3])),
                    encryptor.encrypt(batch_encoder.encode([4])),
                    encryptor.encrypt(batch_encoder.encode([2]))]
result_encrypted = evaluator.multiply_many(encrypted_values, relin_keys)


In [ ]:
result_decrypted = decryptor.decrypt(result_encrypted)
result_decoded = batch_encoder.decode(result_decrypted)
print(result_decoded)


[48  0  0 ...  0  0  0]


## 리스트 가능한지 테스트

In [ ]:
# 리스트 데이터
data_list = [1, 2, 3, 4]

# 인코딩 및 암호화
plain = batch_encoder.encode(data_list)
encrypted = encryptor.encrypt(plain)

# 동형 연산 수행 (예: 제곱)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# 복호화 및 디코딩
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

print(decoded_result)


[1 4 9 ... 0 0 0]


## tuple 가능한지 테스트

In [ ]:
data_tuple = (1, 2, 3, 4)

# Encode and encrypt
plain = batch_encoder.encode(data_tuple)
encrypted = encryptor.encrypt(plain)

# Perform operations (e.g., square)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# Decrypt and decode
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

# Convert list back to tuple if needed
result_tuple = tuple(decoded_result)

print(result_tuple)

(1, 4, 9, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

##  Numpy 사용 가능한지 test
-> 1차원만 가능

In [ ]:
# NumPy 배열 생성
data_array = np.array([1, 2, 3, 4])
array_2x2 = np.array([[1, 2], [3, 4]])

# NumPy 배열을 리스트로 변환
flattened_list = array_2x2.flatten().tolist()

# 인코딩 및 암호화
plain = batch_encoder.encode(flattened_list)
encrypted = encryptor.encrypt(plain)

# 동형 연산 수행 (예: 제곱)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# 복호화 및 디코딩
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

# 결과를 NumPy 배열로 변환
result_array = np.array(decoded_result)
print(result_array)


[1 4 9 ... 0 0 0]
